### Import libraries and extract information from Wikipedia.

In [1]:
#Import libraries
import pandas as pd
import numpy as np
import requests 
from bs4 import BeautifulSoup 

#Wikipedia's URL, connect to the page and get data
URL = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
r = requests.get(URL) 

#Read content
soup = BeautifulSoup(r.content, 'html5lib') 

#Find table with information
table = soup.find('table', attrs = {'class':'wikitable sortable'}) 

#Create an empty list to put data
my_list = []
#Go through table rows and extract text from columns and add into my_list
for idx, row in enumerate(table.findAll('td')):
    x  = row.text
    x = x.replace('\n','')
    my_list.append(x)

#Create chunks fuction to split list into different lists
def chunks(l, n):
    for i in range(0, len(l), n):
        yield l[i:i + n]

#Headers of data frame
header = ['Postcode','Borough', 'Neighbourhood']
#Data of dataframe
data = list(chunks(my_list,3))

#Finally, create my dataframe with table data
df = pd.DataFrame(data,columns=header)

#Remove Not assigned cells
df = df[df['Borough'] != 'Not assigned']
#Group cells by Postcode and Borough and group neighbourhoods using ', ' as delimiter
df = df.groupby(['Postcode','Borough'])['Neighbourhood'].apply(lambda x: ', '.join(x.dropna().unique())).reset_index()
#Find not assigned neighourhoods and if that's the case use the Borough name, otherwise keep the neighbourhood value
df['Neighbourhood'] = np.where(df['Neighbourhood'] == 'Not assigned', df['Borough'],df['Neighbourhood'])

#Shape
df.shape

(103, 3)

### Merge Wikipedia's data with Location of Neighboors

In [2]:
#Import data
path  = 'C:\\Users\\renon\\Downloads\\Geospatial_Coordinates.csv'
geo_data = pd.read_csv(path)

#Merge
df = pd.merge(df,geo_data,left_on='Postcode', right_on='Postal Code', how='left')
df = df[['Postcode', 'Borough', 'Neighbourhood', 'Latitude','Longitude']]
df.columns = ['Postal Code', 'Borough', 'Neighbourhood', 'Latitude','Longitude']

### Create Connection with Foursquare to pull information using the API

In [3]:
#My User ID
CLIENT_ID = 'UOYNHZFOHY5ETVSAPG4NKSH20V3PPSFSZCCRLLT02BRLDUU1' # your Foursquare ID
CLIENT_SECRET = 'OTGN5Q4YMFVIQR3XCV3K4XLJ3H1HFYLIJZJALDVYRGGF3AWK' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100
radius = 500

### Create function to retrieve information using the parameters above. The function will get nearby values based on the Latitude and Longitude of the Neighboors.

In [4]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

### I'm going to fitler my data frame to keep only Boroughs which contains the word 'Toronto'.

In [5]:
#Create toronto df
toronto_df = df[df['Borough'].str.contains("Toronto")]

# type your answer here
toronto_venues = getNearbyVenues(names=toronto_df['Neighbourhood'],
                                   latitudes=toronto_df['Latitude'],
                                   longitudes=toronto_df['Longitude']
                                  )

The Beaches
The Danforth West, Riverdale
The Beaches West, India Bazaar
Studio District
Lawrence Park
Davisville North
North Toronto West
Davisville
Moore Park, Summerhill East
Deer Park, Forest Hill SE, Rathnelly, South Hill, Summerhill West
Rosedale
Cabbagetown, St. James Town
Church and Wellesley
Harbourfront, Regent Park
Ryerson, Garden District
St. James Town
Berczy Park
Central Bay Street
Adelaide, King, Richmond
Harbourfront East, Toronto Islands, Union Station
Design Exchange, Toronto Dominion Centre
Commerce Court, Victoria Hotel
Roselawn
Forest Hill North, Forest Hill West
The Annex, North Midtown, Yorkville
Harbord, University of Toronto
Chinatown, Grange Park, Kensington Market
CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara
Stn A PO Boxes 25 The Esplanade
First Canadian Place, Underground city
Christie
Dovercourt Village, Dufferin
Little Portugal, Trinity
Brockton, Exhibition Place, Parkdale Village
High Park, The 

### Create data frame to run KMeans and perform a Clustering Analysis

In [13]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Coffee Shop,Café,Steakhouse,Bar,American Restaurant,Thai Restaurant,Gym,Sushi Restaurant,Bakery,Restaurant
1,Berczy Park,Coffee Shop,Cocktail Bar,Bakery,Farmers Market,Beer Bar,Steakhouse,Cheese Shop,Café,Seafood Restaurant,Basketball Stadium
2,"Brockton, Exhibition Place, Parkdale Village",Yoga Studio,Breakfast Spot,Café,Coffee Shop,Bakery,Stadium,Burrito Place,Restaurant,Caribbean Restaurant,Climbing Gym
3,Business Reply Mail Processing Centre 969 Eastern,Yoga Studio,Auto Workshop,Smoke Shop,Skate Park,Brewery,Burrito Place,Butcher,Restaurant,Recording Studio,Park
4,"CN Tower, Bathurst Quay, Island airport, Harbo...",Airport Service,Airport Lounge,Airport Terminal,Bar,Boutique,Airport,Airport Food Court,Airport Gate,Boat or Ferry,Harbor / Marina


### Next I'm going to run a clustering analysis using KMeans, the number of clusters will be 5. Basically the algorithm will start grouping neighboors based on the Category of the venues.

In [14]:
from sklearn.cluster import KMeans

# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_venues

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,The Beaches,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail,2,Health Food Store,Other Great Outdoors,Trail,Pub,Wings Joint,Donut Shop,Discount Store,Dive Bar,Dog Run,Doner Restaurant
1,The Beaches,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store,2,Health Food Store,Other Great Outdoors,Trail,Pub,Wings Joint,Donut Shop,Discount Store,Dive Bar,Dog Run,Doner Restaurant
2,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub,2,Health Food Store,Other Great Outdoors,Trail,Pub,Wings Joint,Donut Shop,Discount Store,Dive Bar,Dog Run,Doner Restaurant
3,The Beaches,43.676357,-79.293031,Glen Stewart Ravine,43.676300,-79.294784,Other Great Outdoors,2,Health Food Store,Other Great Outdoors,Trail,Pub,Wings Joint,Donut Shop,Discount Store,Dive Bar,Dog Run,Doner Restaurant
4,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood,2,Health Food Store,Other Great Outdoors,Trail,Pub,Wings Joint,Donut Shop,Discount Store,Dive Bar,Dog Run,Doner Restaurant


### Next, I'm going to plot the results in a map so we can see how the clusters are distributed.

In [15]:
import folium
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import matplotlib.cm as cm
import matplotlib.colors as colors

# create map
map_clusters = folium.Map(location=[43.676357, -79.293031], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Neighborhood Latitude'], toronto_merged['Neighborhood Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Analyze Clusters

In [16]:
df_analyze = toronto_merged[['Neighborhood', 'Cluster Labels', '1st Most Common Venue', '2nd Most Common Venue',
       '3rd Most Common Venue', '4th Most Common Venue',
       '5th Most Common Venue', '6th Most Common Venue',
       '7th Most Common Venue', '8th Most Common Venue',
       '9th Most Common Venue', '10th Most Common Venue']].drop_duplicates()

df_analyze[df_analyze['Cluster Labels'] == 0]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
175,"Moore Park, Summerhill East",0,Playground,Restaurant,Dim Sum Restaurant,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop


In [17]:
df_analyze[df_analyze['Cluster Labels'] == 1]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
46,"The Beaches West, India Bazaar",1,Park,Sandwich Place,Board Shop,Pub,Liquor Store,Burger Joint,Burrito Place,Fast Food Restaurant,Fish & Chips Shop,Steakhouse
114,Davisville North,1,Gym,Clothing Store,Breakfast Spot,Food & Drink Shop,Sandwich Place,Hotel,Asian Restaurant,Park,Grocery Store,Pizza Place
192,Rosedale,1,Park,Playground,Trail,Building,Dim Sum Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant
1127,"Forest Hill North, Forest Hill West",1,Bus Line,Sushi Restaurant,Park,Jewelry Store,Trail,Doner Restaurant,Discount Store,Dive Bar,Dog Run,Wings Joint


In [18]:
df_analyze[df_analyze['Cluster Labels'] == 2]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,The Beaches,2,Health Food Store,Other Great Outdoors,Trail,Pub,Wings Joint,Donut Shop,Discount Store,Dive Bar,Dog Run,Doner Restaurant
5,"The Danforth West, Riverdale",2,Greek Restaurant,Coffee Shop,Italian Restaurant,Ice Cream Shop,Furniture / Home Store,Liquor Store,Bakery,Sports Bar,Spa,Juice Bar
69,Studio District,2,Café,Coffee Shop,Italian Restaurant,Gastropub,American Restaurant,Bakery,Yoga Studio,Bar,Fish Market,Coworking Space
124,North Toronto West,2,Clothing Store,Coffee Shop,Health & Beauty Service,Sporting Goods Shop,Chinese Restaurant,Dessert Shop,Diner,Fast Food Restaurant,Italian Restaurant,Mexican Restaurant
143,Davisville,2,Dessert Shop,Sandwich Place,Pizza Place,Italian Restaurant,Sushi Restaurant,Coffee Shop,Café,Gourmet Shop,Farmers Market,Restaurant
177,"Deer Park, Forest Hill SE, Rathnelly, South Hi...",2,Coffee Shop,Pub,Pizza Place,American Restaurant,Light Rail Station,Sports Bar,Bagel Shop,Restaurant,Supermarket,Sushi Restaurant
197,"Cabbagetown, St. James Town",2,Coffee Shop,Bakery,Café,Pizza Place,Restaurant,Market,Italian Restaurant,Pet Store,Pub,Bank
242,Church and Wellesley,2,Coffee Shop,Japanese Restaurant,Gay Bar,Sushi Restaurant,Restaurant,Pub,Men's Store,Mediterranean Restaurant,Hotel,Gym
330,"Harbourfront, Regent Park",2,Coffee Shop,Bakery,Pub,Park,Breakfast Spot,Mexican Restaurant,Theater,Restaurant,Gym / Fitness Center,Café
379,"Ryerson, Garden District",2,Coffee Shop,Clothing Store,Cosmetics Shop,Café,Middle Eastern Restaurant,Fast Food Restaurant,Diner,Plaza,Pizza Place,Ramen Restaurant


In [19]:
df_analyze[df_analyze['Cluster Labels'] == 3]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
110,Lawrence Park,3,Gym / Fitness Center,Park,Swim School,Bus Line,Diner,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant


In [20]:
df_analyze[df_analyze['Cluster Labels'] == 4]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1125,Roselawn,4,Garden,Home Service,Dim Sum Restaurant,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop


## Clusters:
- 0: Close to a Playground and several restaurants.
- 1: Close to Parks, Stores and some restaurants.
- 2: Close to several Coffe Shops, Bars and restaurants.
- 3: Close to Gym, Park and Swim school.
- 4: CLose to a Garden and restaurants.